# Auto-Enable Flow Logs

This will kick off templates that will auto-enable VPC flow logs across the organization.

In [1]:
from jupyterirtools import sso
import os
import time
import importlib
import json

sso.login("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])

If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=SRBW-JCQD to activate the session


<IPython.core.display.Javascript object>

### Prerequisite: Create a flow logs bucket
A destination bucket needs to exist before the flow logs can be added. Typically this is a bucket in the logging account.

Options:
* Option 1: Choose Existing Bucket
  * Find an existing bucket with a bucket policy that allows cloudwatch logs to write to the bucket.
  * Save the bucket name to the SSM Parameter 'Jupyter-Flowlogs-Bucket' in the management account.
* Option 2: Manually create the bucket
  * Create a bucket with the appropriate permissions
  * Save the bucket name to the SSM Parameter 'Jupyter-Flowlogs-Bucket' in the management account.
* Option 3: Run the create logging bucket notebook
  * Run the [create logging notebook](create_flowlogs_bucket.ipynb):



In [4]:
# Validate the SSM Parameters exists for 'Jupyter-Flowlogs-Bucket' in the management account.
session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])
ssm_client = session.client('ssm')

try:
    flowlogs_bucket_response = ssm_client.get_parameter(Name='Jupyter-Flowlogs-Bucket')
    flowlogs_bucket = flowlogs_bucket_response['Parameter']['Value']
    print(f"flowlogs_bucket={flowlogs_bucket} configured. Continue to the next step.")
except ssm_client.exceptions.ParameterNotFound:
    print("Parameter Jupyter-Flowlogs-Bucket not found. Confirm configuration of the flow logs bucket.")

    


flowlogs_bucket=flowlogs-bucket-cfncloudtraillogsbucket-1be4gsnc376f0 configured. Continue to the next step.


In [5]:
from jupyterirtools import cloudformation
import importlib
import boto3
importlib.reload(cloudformation)

org_client = boto3.client('organizations')
desc_client = org_client.describe_organization()
org_unit_id = org_client.list_roots()['Roots'][0]['Id']

org_id = desc_client['Organization']['Id']

cfn_bucket_stack = cloudformation.deploy(stack_name="cfn-bucket-stack", 
                      role="Jupyter-IR-AdministratorAccess", 
                      account=os.environ['MANAGEMENT_ACCOUNT'], 
                      template="../cfn-templates/cfn-bucket.yaml",
                      parameters={
                          "OrgId": org_id
                      })

bucket_name = cfn_bucket_stack['Outputs']['CfnBucket']

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


aws cloudformation deploy --stack-name=cfn-bucket-stack --template-file ../cfn-templates/cfn-bucket.yaml --capabilities CAPABILITY_IAM --profile Jupyter-IR-AdministratorAccess-383086473915 --parameter-overrides  OrgId=o-krcf45heml
Before readLine()
After readLine()
2

After readLine()
After readLine()
Waiting for changeset to be created..
After readLine()
After readLine()

After readLine()
After readLine()
No changes to deploy. Stack cfn-bucket-stack is up to date
After readLine()
After readLine()
Before read()
After read()
Before readLine()
After readLine()
3
Before read()
After read()


In [44]:
from jupyterirtools import cloudformation
import random, string
importlib.reload(cloudformation)

cfn_profile = f"Jupyter-IR-AdministratorAccess-{os.environ['MANAGEMENT_ACCOUNT']}"
session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])

random_str=''.join(random.choices(string.ascii_uppercase + string.digits, k=8))
!aws s3 sync ../cfn-templates/ s3://$bucket_name/$random_str/ --profile $cfn_profile > /dev/null

template_url = f'https://s3.us-east-1.amazonaws.com/{bucket_name}/{random_str}'

In [45]:
supported_regions = ["us-east-1", "us-east-2","us-west-1","us-west-2","ap-east-1","ap-south-1","ap-northeast-2",
"ap-southeast-1","ap-southeast-2","ap-northeast-1", "ca-central-1","cn-north-1","cn-northwest-1",
"eu-north-1","eu-central-1","eu-west-1","eu-west-2","eu-west-3","me-south-1","me-central-1",
"sa-east-1", "us-gov-east-1", "us-gov-west-1"]

In [46]:
regions = []

ec2_client = session.client('ec2')
region_response = ec2_client.describe_regions()

for region in region_response['Regions']:
    if region['RegionName'] in supported_regions:
        regions.append(region['RegionName'])

        
session = sso.get_session("Jupyter-IR-AdministratorAccess", os.environ['MANAGEMENT_ACCOUNT'])
ssm_client = session.client('ssm')

flowlogs_bucket_response = ssm_client.get_parameter(Name='Jupyter-Flowlogs-Bucket')
flowlogs_bucket = flowlogs_bucket_response['Parameter']['Value']
    
flow_log_remediation_stack = cloudformation.deploy(stack_name="jupyter-ir-config", 
                      role="Jupyter-IR-AdministratorAccess", 
                      account=os.environ['MANAGEMENT_ACCOUNT'], 
                      template="../cfn-templates/organization-ir-config.yaml",
                      parameters={
                          "FlowLogsBucket": flowlogs_bucket,
                          "AutomateRemediationTemplateUrl": template_url,
                          "OrgUnitId": org_unit_id,
                          "OutputFormat": "parquet",
                          "RegionList": ','.join(regions)
                      })

flow_log_remediation_stack

aws cloudformation deploy --stack-name=jupyter-ir-config --template-file ../cfn-templates/organization-ir-config.yaml --capabilities CAPABILITY_IAM --profile Jupyter-IR-AdministratorAccess-383086473915 --parameter-overrides  FlowLogsBucket=flowlogs-bucket-cfncloudtraillogsbucket-1be4gsnc376f0 AutomateRemediationTemplateUrl=https://s3.us-east-1.amazonaws.com/cfn-bucket-stack-cfnbucket-1l1tf6muc97gf/00CJDM3Z OrgUnitId=r-nvfg OutputFormat=parquet RegionList=eu-north-1,ap-south-1,eu-west-3,eu-west-2,eu-west-1,ap-northeast-2,ap-northeast-1,sa-east-1,ca-central-1,ap-southeast-1,ap-southeast-2,eu-central-1,us-east-1,us-east-2,us-west-1,us-west-2

Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - jupyter-ir-config


{'Name': 'jupyter-ir-config',
 'Outputs': {'FlowLogsBucket': 'flowlogs-bucket-cfncloudtraillogsbucket-1be4gsnc376f0'},
 'Status': 'UPDATE_COMPLETE'}